# `HF_classification . blanchefort . rubert-base-cased-sentiment-rusentiment . LogReg` notebook
### Descriprion
This notebook is one of those which were used to training large DL models from `transformers` library and measuring classification performance. Here the model **rubert-base** was trained. Then it provided Logistic regression classifier with embeddings for reviews (documents, in terms of NLP). Then LogisticRegression model was trained with these representations and did not show results better than Linear layer from PyTorch.

In [322]:
#!g1.1
%pwd

'/home/jupyter/work/resources/nlp-coursework'

In [288]:
%cd ../../..

/home/jupyter/work/resources/nlp-coursework


In [323]:
#!g1.1
from datasets_ import DatasetLoader
from src.nlp.application import BertLogRegClassifier, ReviewDataset

from datasets import load_dataset

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.utils import resample

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import AdamW

from tqdm import tqdm
import numpy as np
import pandas as pd

import sys
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

from nltk import WhitespaceTokenizer

from IPython.display import clear_output

from transformers import (
    pipeline,                       
    AutoModelForSequenceClassification,                       
    BertForSequenceClassification,                       
    AutoTokenizer,
    AdamW,
)

In [324]:
#!g1.1
train, test = DatasetLoader.load_reviews_Review_Label_dataset(train_test_split=True,
                                                              classnames_to_int=True,
                                                              remove_neutral_class=True,
                                                              show_path=True,)
train.label[train['label'] == 2] = 1
test.label[test['label'] == 2] = 1

print(train.label.value_counts())
train = pd.concat([train, resample(train[train.label == 0], n_samples=41608, random_state=42)])
print(train.label.value_counts())

print(train.shape, test.shape)

/home/jupyter/work/resources/nlp-coursework/data/reviews_Review_Label/reviews_Review_Label.csv
label
1    48477
0     6869
Name: count, dtype: int64
label
0    48477
1    48477
Name: count, dtype: int64
(96954, 2) (23721, 2)


<ipython-input-3-8adfedff2195>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.label[train['label'] == 2] = 1
<ipython-input-3-8adfedff2195>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.label[test['label'] == 2] = 1


In [331]:
#!g1.1
# clf = BertLogRegClassifier('blanchefort/rubert-base-cased-sentiment-rusentiment')

train_dataset = ReviewDataset(train.review, train.label, clf.tokenizer)
test_dataset = ReviewDataset(test.review, test.label, clf.tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [332]:
#!g1.1
for param in clf.model.parameters():
    param.requires_grad = False
print('All parameters:', sum(p.numel() for p in clf.model.parameters()))
print('Trainable parameters:', sum(p.numel() for p in clf.model.parameters() if p.requires_grad))

All parameters: 177853440
Trainable parameters: 0


In [333]:
#!g1.1
clf.fit(train_dataloader)

  0%|                                                                      | 0/1515 [00:00<?, ?it/s]

In [328]:
#!g1.1
clf.all_train_embeddings.shape

(100, 768)

In [334]:
#!g1.1
clf.predict(test_dataloader, print_report=True)

  0%|                                                                       | 0/371 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.57      0.90      0.70      2979
           1       0.98      0.90      0.94     20742

    accuracy                           0.90     23721
   macro avg       0.78      0.90      0.82     23721
weighted avg       0.93      0.90      0.91     23721



array([1, 0, 0, ..., 1, 0, 1])

In [336]:
#!g1.1
import dill

with open('models/clf.model', 'rb') as f:
    pretrained_nn_clf = dill.load(f)

In [337]:
#!g1.1
clf.model = pretrained_nn_clf.model

In [339]:
#!g1.1
clf.model.dropout = torch.nn.Sequential()
clf.model.classifier = torch.nn.Sequential()

In [341]:
#!g1.1
preds = clf.predict(test_dataloader, print_report=True)

  0%|                                                                       | 0/371 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.77      0.83      0.80      2979
           1       0.98      0.96      0.97     20742

    accuracy                           0.95     23721
   macro avg       0.87      0.90      0.89     23721
weighted avg       0.95      0.95      0.95     23721

